Analise de Sentimento realtime no Twitter

** Configurações utilizadas **

Apache Spark *** / Python 3.2* / Ubunto **

Vamos utilizar Framework Spark Streaming para processar dados do Twitter

In [ ]:
# Sera necessario instalar esses pacotes
#!pip install request_oauthlib # Pacote para autenticacao
!pip install twython # Pacote para fazer a coleta das informacoes do twitter
!pip install nltk # Pacote de processamento de linguagem natural.
!pip install requests requests_oauthlib # Caso de erro no comando 1.

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-209902e4-f64a-4c95-9d7b-df9dd10cf1a7/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-209902e4-f64a-4c95-9d7b-df9dd10cf1a7/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Modulos usados

from pyspark.streaming import StreamingContext # Aplicacao para trabalhar dentro do spark
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json
import pylab as pl

In [ ]:
# Pacote NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [ ]:
# Frequencia de update - a cada 5 segundos grava os resultados coletados do twitter
INTERVALO_BATCH = 5

In [ ]:
# Criando o StreamingContext - Contexto de manipulacao do Streaming de dados
ssc = StreamingContext(sc, INTERVALO_BATCH)

Treinando o Classificador de Analise de Sentimento

Uma parte importante deste case para a criação do algoritmo analise de sentimento(ou qualquer algoritmo de mineração de dados) é ter um conjunto de dados para treino(corpus), para que seu algoritmo saiba classificar e atender os padrões de precisão que você espera. Para isso foi utilizado como base o dataset de treino fornecido pela kaggle. https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis

Aqui vamos classificar se o twitter teve um sentimento bom ou ruim. Classificando-o como 0 e 1.

0 para sentimento negativo

1 para sentimento positivo

In [ ]:
# Lendo o arquivo texto e criando um RDD em memória com Spark
arquivo = sc.textFile("/FileStore/tables/datatraining/Train100.csv")

In [ ]:
# Removendo o cabecalho
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line: line != header)

In [ ]:
# Confirmando o tipo de dataset
type(dataset)

Out[15]: pyspark.rdd.PipelinedRDD

In [ ]:
# Função separa as conlunas de cada linha, cria uma tupla e remove a pontuação. Faz limpeza de dados
def get_row(line):
    row = line.split(';')
    sentimento = row[1]
    tweet = row[3].strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split (' ')
    tweet_lower = []
    for word in tweet:
        tweet_lower.append(word.lower())
    return (tweet_lower, sentimento)

In [ ]:
# Aplica a função a cada linha do dataset
dataset_treino = dataset.map(lambda line: get_row(line))

In [ ]:
# Cria um objeto SentimentAnalyzer
sentiment_analyzer = SentimentAnalyzer()

In [ ]:
# Certifique de ter espaço em disco 
# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
#nltk.download() # baixar tudo do nltk
nltk.download()
nltk.download('stopwords')

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


---------------------------------------------------------------------------
StdinNotImplementedError                  Traceback (most recent call last)
<command-3527870223010453> in <cell line: 4>()
      2 # https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
      3 #nltk.download() # baixar tudo do nltk
----> 4 nltk.download()
      5 nltk.download('stopwords')

/local_disk0/.ephemeral_nfs/envs/pythonEnv-209902e4-f64a-4c95-9d7b-df9dd10cf1a7/lib/python3.9/site-packages/nltk/downloader.py in download(self, info_or_id, download_dir, quiet, force, prefix, halt_on_error, raise_on_error, print_error_to)
    761             if download_dir is not None:
    762                 self._download_dir = download_dir
--> 763             self._interactive_download()
    764             return True
    765 

/local_disk0/.ephemeral_nfs/envs/pythonEnv-209902e4-f64a-4c95-9d7b-df9dd10cf1a7/lib/python3.9/site-packages/nltk/downloader.py in _interactive_download(self)
   1115             

In [ ]:
from nltk.corpus import stopwords

In [ ]:
# Obtem a lista de stopwords em portugues
stopwords_all = []
for word in stopwords.words('portuguese'):
    stopwords_all.append(word)
    stopwords_all.append(word + '_NEG')

In [ ]:
# Obtenado 20.000 tweets do dataset de treino e retorna todas as palavras que não são stopwords
dataset_treino_amostra = dataset_treino.take(20000)

In [ ]:
dataset_treino_amostra[1]

Out[52]: (['1'],
 '@lalenes_ O bom de lembrar é pra não repetir, com a próxima pessoa :D')

In [ ]:
print (dataset_treino_amostra)

[(['1'], 'FAV e falo de vocês por dm :)'), (['1'], '@lalenes_ O bom de lembrar é pra não repetir, com a próxima pessoa :D'), (['1'], 'vamos porquinhos !!! :D https://t.co/UFwKt1dCxe'), (['1'], 'Tenho raiva de vc por vc ser tão linda, eu me apaixono fácil hehehehe — kkkk obg :) https://t.co/0OBt5PKmU9'), (['1'], '@Daniel_Albino Eu disse espectacular :)'), (['1'], 'Feliz por meu brownie ter ficado bom :)'), (['1'], 'Manda alguma coisa por favor :) https://t.co/Fen4dst7EF'), (['1'], 'Lindo dia :) https://t.co/eLOrjqwbqS'), (['1'], '@NunoInverno Gosto de cozinhar sim :)'), (['1'], '@bielconn Sim biel , se prepara pra sair nego até dos mortos pra ter cartoon gratis :D'), (['1'], '@fjvne Torço muito pra que o Vasco se recupere, A PARTIR DE HOJE, vocês não merecem esse sofrimento. Saudações palestrinas, torcida cruzmaltina. :)'), (['1'], '@Sonykok parece boa :) https://t.co/DiW45ECtKu'), (['1'], 'vou deixar o relatório 9 pra fazer depois como todos os outros :)'), (['1'], 'além de linda é mui

In [ ]:
dataset_treino_amostra[1]

Out[63]: (['1'],
 '@lalenes_ O bom de lembrar é pra não repetir, com a próxima pessoa :D')

In [ ]:
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all] #retirando as informações que não são stopwords função {list comprehension range}.
# List Comprehensions são maneiras de criar e manipular listas dentro do python de forma mais curta em apenas 1 linha. 
# Geralmente é utilizado para realizar operações simples dentro de uma lista (array). Permite você criar e processar listas de forma limpa e fácil.

In [ ]:
# Criando um unigram (n-grama) e extrair as features (ajudar a entender o sentido da frase, se é bom, ruim etc.)
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg_nostops, top_n = 200)
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = sentiment_analyzer.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

Out[57]: nltk.collections.LazyMap

In [ ]:
print(training_set)

[({'contains(1)': True, 'contains(sep)': False, 'contains(0000)': False, 'contains(2018)': False, 'contains(wed)': False, 'contains(05)': False, 'contains(sun)': False, 'contains(09)': False, 'contains(thu)': False, 'contains(06)': False, 'contains()': False, 'contains(mon)': False, 'contains(03)': False, 'contains(sat)': False, 'contains(08)': False, 'contains(tue)': False, 'contains(04)': False, 'contains(de)': False, 'contains(eu)': False, 'contains(3)': False, 'contains(p)': False, 'contains(que)': False, 'contains(fri)': False, 'contains(07)': False, 'contains(lt)': False, 'contains(gt)': False, 'contains(q)': False, 'contains(se)': False, 'contains(e)': False, 'contains(mais)': False, 'contains(bom)': False, 'contains(pode)': False, 'contains(te)': False, 'contains(mas)': False, 'contains(vejo)': False, 'contains(pra)': False, 'contains(hoje)': False, 'contains(não)': False, 'contains(é)': False, 'contains(tenho)': False, 'contains(falar)': False, 'contains(com)': False, 'contain

In [ ]:
# Treinar o modelo
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)

Training classifier


In [ ]:
# Testando o classificador em algumas sentenças
test_sentence1 = [(['este', 'banco', 'é', 'ruim'], '')]
test_sentence2 = [(['hoje', 'foi', 'um', 'dia', 'dificil'], '')]
test_sentence3 = [(['bom', 'maravilhoso', 'legal', 'gostei'], '')]
test_set1 = sentiment_analyzer.apply_features(test_sentence1)
test_set2 = sentiment_analyzer.apply_features(test_sentence2)
test_set3 = sentiment_analyzer.apply_features(test_sentence3)

In [ ]:
test_set3

Out[81]: [({'contains(1)': False, 'contains(sep)': False, 'contains(0000)': False, 'contains(2018)': False, 'contains(wed)': False, 'contains(05)': False, 'contains(sun)': False, 'contains(09)': False, 'contains(thu)': False, 'contains(06)': False, 'contains()': False, 'contains(mon)': False, 'contains(03)': False, 'contains(sat)': False, 'contains(08)': False, 'contains(tue)': False, 'contains(04)': False, 'contains(3)': False, 'contains(p)': False, 'contains(fri)': False, 'contains(07)': False, 'contains(lt)': False, 'contains(d)': False, 'contains(gt)': False, 'contains(q)': False, 'contains(bom)': True, 'contains(pode)': False, 'contains(vejo)': False, 'contains(pra)': False, 'contains(hoje)': False, 'contains(falar)': False, 'contains(vou)': False, 'contains(querer)': False, 'contains(dps)': False, 'contains(vc)': False, 'contains(_NEG)': False, 'contains(httpstcoapwbkroxlw)': False, 'contains(falo)': False, 'contains(sim)': False, 'contains(celularamp)': False, 'contains(100)': F

Conectando no Twitter, utilizando login de desenvolvedor.
Como criar conta de desenvolver? 

Conforme link abaixo:

https://medium.com/programadores-ajudando-programadores/api-do-twitter-criando-o-app-e-obtendo-os-tokens-28ef3e2a281c

In [ ]:
# Autenticação do Twitter, necessario gerar a alteticação no modo conta de desenvolvedor {LINK de como fazer}
consumer_key = "xxxx"
consumer_secret = "xxxx"
access_token = "xxxx"
access_token_secret = "xxxx"

In [ ]:
# Especifica a URL, termo de busca (Twitter abre uma porta para os desenvolvedores, por isso a URL abaixo)
search_term = 'Santander'
sample_url = 'https://stream.twitter.com/1.1/statuses/sample.json'
filter_url = 'https://stream.twitter.com/1.1/statuses/filter.json?track='+search_term

In [ ]:
# Criando o objeto de autenticação para o Twitter
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)

In [ ]:
# Configurando o Stream, criando fila
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([], default = rdd)

In [ ]:
type(stream)

Out[93]: pyspark.streaming.dstream.DStream

In [ ]:
# Total de tweets por update
NUM_TWEETS = 500

In [ ]:
# Função conecta ao Twitter e retorna um numero especifico de Tweets (NUM_TWEETS)
def tfunct(t, rdd):
    return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
    response = resquest.get(filter_url, auth = auth, stream = True)
    print(filter_url, response)
    count = 0
    for line in response.iter_lines():
        try:
            if count > NUM_TWEETS:
                break
            post = json.loads(line.decode('utf-8'))
            contents =[post['text']]
            count += 1
            yield str(contents)
        except:
            result = False

In [ ]:
stream = stream.transform(tfunct)

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
# Essa função classifica os tweets, aplicando as fesatures do modelo criado anteriormente
def classifica_tweet(tweet):
    sentence = [(tweet, '')]
    test_set = sentiment_analyzer.apply_features(sentence)
    print(tweet, classifier.classify(test_set[0][0]))
    return(tweet, classifier.classify(test_set[0][0]))

In [ ]:
# Função para retonar o texto do Twitter
def get_tweet_text(rdd):
    for line in rdd:
        tweet = line.strip()
        translator = str.maketrans({key: None for key in string.punctuation})
        tweet = tweet.translate(translator)
        tweet = tweet.split(' ')
        tweet_lower = []
        for word in tweet:
            tweet_lower.append(word.lower())
        return(classifca_tweet(tweet_lower))

In [ ]:
# Cria uma lista vazia para os resultados
resultados = []

In [ ]:
# Essa função salva o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
    global resultados
    pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
    counts = pairs.reduceBykey(add)
    output = []
    for count in counts.collect():
        output.append(count)
    result = [time.strftime("%I:%M:%S"), output]
    resultados.append(result)
    print(result)

In [ ]:
# A função foreachRDD() aplica uma função a cada RDD to streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
# Start streaming
ssc.start()
#scc.awaitTermination()

In [ ]:
# Mostrar o resultado maior que 5
cont = True
while cont:
    if len(resultados) > 5:
        cont = False

In [ ]:
# Grava os resultados
rdd_save = '/FileStore/tables/santander/r'+time.strftime("%I,%M%S")
resultados_rdd = sc.parallelize(resultados)
resultados_rdd.saveAsTextFile(rdd_save)

In [ ]:
# Visualiza os resultados
resultados_rdd.collect()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-3527870223010480> in <cell line: 2>()
      1 # Visualiza os resultados
----> 2 resultados_rdd.collect()

NameError: name 'resultados_rdd' is not defined

In [ ]:
# Finaliza o Streaming
ssc.stop()